In [8]:
import pandas as pd
from osmapi import OsmApi
MyApi = OsmApi()
import pyproj
#import polars as pl
pd.set_option('display.max_columns', 500)

pd.options.mode.chained_assignment = None  # default='warn'

geodesic = pyproj.Geod(ellps='WGS84')

selected_cols = ['data.id','data.timestamp','data.user','data.uid','data.tag.railway','data.tag.highway','data.tag.service','data.tag.tiger:county','data.tag.name','data.tag.electrified','data.tag.gauge','data.tag.operator','data.tag.owner','data.tag.railway:preserved','data.tag.usage','data.tag.maxspeed','prev_node','prev_node_lat','prev_node_lon','fwd_azimuth_to_crossing','back_azimuth_to_crossing','distance_to_crossing','related__OSM_Node','related__OSM_Node_lat','related__OSM_Node_lon','realted_FRA_ID','data.tag.access','data.tag.surface','data.tag.bicycle']

In [2]:
#Cast Date Column of InventoryDf
inventoryDf = pd.read_excel('Crossing_CA.xlsx')
inventoryDf['revisiondate'] = pd.to_datetime(inventoryDf['revisiondate'])

#Sort and Drop duplicates


inventoryDf = inventoryDf.sort_values('revisiondate').drop_duplicates('crossingid',keep='last')
#df.sort_values('DATE_CHANGED').drop_duplicates('STATION_ID',keep='last')
inventoryDf = inventoryDf[inventoryDf['latitude'].notnull()]

inventoryDf = inventoryDf[inventoryDf['longitude'].notnull()]

In [3]:
from collections.abc import MutableMapping

def flatten_dict(d: MutableMapping, sep: str= '.') -> MutableMapping:
    [flat_dict] = pd.json_normalize(d, sep=sep).to_dict(orient='records')
    return flat_dict

In [9]:
allRows = []


count = 0

for i, row in inventoryDf.iloc[1:10].iterrows():
    
    print(count, ' FRA: ', row['crossingid'])
    count = count + 1
    
    try:
    
        fra_crossing_id = row['crossingid']
        fra_lat = row['latitude']
        fra_lon = row['longitude']
        
        

        search_level = 0.001
        max_search_level = 0.005
        
        results_dict = MyApi.Map(fra_lon-search_level,fra_lat-search_level,fra_lon+search_level,fra_lat+search_level)
        
        flat_dict = []
        for d in results_dict:
            flat_dict.append(flatten_dict(d))

        df = pd.DataFrame.from_dict(flat_dict)
        
        crossings_df = pd.DataFrame()
        
        if('data.tag.railway' in df.columns):
            crossings_df= df[df['data.tag.railway'] == 'level_crossing']
         
        
        while search_level < max_search_level and crossings_df.shape[0] <= 0:
            search_level = search_level* 2
            print('search area increased to ', search_level)
            results_dict = MyApi.Map(fra_lon-search_level,fra_lat-search_level,fra_lon+search_level,fra_lat+search_level)
        
            flat_dict = []
            for d in results_dict:
                flat_dict.append(flatten_dict(d))

            df = pd.DataFrame.from_dict(flat_dict)
            
            if('data.tag.railway' in df.columns):
                crossings_df= df[df['data.tag.railway'] == 'level_crossing']
            
            
        if('data.tag.railway' not in df.columns):
            print("still missing rail")
            continue
        
        if(crossings_df.shape[0] <= 0):
            print("still missing crossing")
            continue


       


        crossings_df['customdistance']= crossings_df.apply(lambda g: geodesic.inv(fra_lon, fra_lat, g['data.lon'], g['data.lat'])[2],axis=1)
        crossings_df = crossings_df.sort_values(by=['customdistance'])

        crossing_node = crossings_df.iloc[0]['data.id']

        osm_crossing_lat = crossings_df.iloc[0]['data.lat']
        osm_crossing_lon = crossings_df.iloc[0]['data.lon']

        ways = df[df['data.nd'].notnull()]
        inWays = ways[ways['data.nd'].apply(lambda x: crossing_node in x)]


        inWays['prev_node'] = inWays['data.nd'].apply(lambda x: x[x.index(crossing_node)-1])

        #get data on the previous way
        inWays['prev_node_lat'] = inWays.apply(lambda x: MyApi.NodeGet(x['prev_node'])['lat'],axis=1)
        inWays['prev_node_lon'] = inWays.apply(lambda x: MyApi.NodeGet(x['prev_node'])['lon'],axis=1)


        inWays['fwd_azimuth_to_crossing']= inWays.apply(lambda g: geodesic.inv(osm_crossing_lon, osm_crossing_lat, g['prev_node_lon'], g['prev_node_lat'])[0],axis=1)
        inWays['back_azimuth_to_crossing']= inWays.apply(lambda g: geodesic.inv(osm_crossing_lon, osm_crossing_lat, g['prev_node_lon'], g['prev_node_lat'])[1],axis=1)
        inWays['distance_to_crossing']= inWays.apply(lambda g: geodesic.inv(osm_crossing_lon, osm_crossing_lat, g['prev_node_lon'], g['prev_node_lat'])[2],axis=1)

        inWays = inWays.assign(related__OSM_Node=crossing_node)


        inWays = inWays.assign(related__OSM_Node_lat=osm_crossing_lat)
        inWays = inWays.assign(related__OSM_Node_lon=osm_crossing_lon)

        inWays = inWays.assign(realted_FRA_ID=fra_crossing_id)
        
            
        
        for iS, rowS in inWays.iterrows():
            allRows.append(rowS.to_dict())
        
           # print(inWays.to_dict())
        
        #every 1000 lets update the csv
        if count % 1000 == 0:
            print("Update CSV ", count)
            df = pd.DataFrame(allRows)
            

            
    except Exception  as e:
        print('something went wrong with :' , i,row['crossingid'], e)
    
    

    
df = pd.DataFrame(allRows)



0  FRA:  751386X
False
something went wrong with : 455 751386X "['poop', 'data.tag.service', 'data.tag.maxspeed', 'data.tag.access', 'data.tag.surface'] not in index"
1  FRA:  857915A
False
something went wrong with : 454 857915A "['poop', 'data.tag.railway:preserved', 'data.tag.bicycle'] not in index"
2  FRA:  498838K
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
3  FRA:  751443J
False
something went wrong with : 452 751443J "['poop', 'data.tag.service', 'data.tag.maxspeed', 'data.tag.surface'] not in index"
4  FRA:  749386X
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
5  FRA:  498982C
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing rail
6  FRA:  847372J
False
something went wrong with : 448 847372J "['poop', 'data.tag.service', 'data.tag.electrified', 'data.tag.gauge', 'data.tag

In [ ]:
import time  

time_stamp = time.time()

file_name = 'CA_AZ_DATA_thread_2' + str(time_stamp) + '.csv'

df.to_csv(file_name)